In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten
from keras.callbacks import EarlyStopping

from loader import load_20news
from custom_layer import MonteCarloLRF, SeparableMonteCarloLRF, SeparableMonteCarloMaxPooling

import numpy as np
import sklearn as sk
import pickle

Using TensorFlow backend.


In [3]:
top_words=10000
sparse=False
remove_short_documents=True
notebook = 'mcNet_top10k_temptative_1'
batch_size = 16

In [4]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=top_words, sparse=sparse, remove_short_documents=remove_short_documents, verbose=False)

In [5]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
num_classes = Y_train.shape[1]

In [6]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(10168, 10000, 1)
(7071, 10000, 1)
16 20


In [7]:
#Process next cell only once
path = os.path.join('probabilities_' + 
                    'top' + str(top_words) +
                    '_sparse' + str(sparse) +
                    '_removeShorts' + str(remove_short_documents) +
                    '.pkl')
if os.path.isfile(path):
  probabilities = pickle.load(open(path, "rb"))

In [8]:
if not(os.path.isfile(path)):
  METRIC = 'cosine'#'euclidean'
  distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

  # enforce exact zero
  for k in xrange(distances.shape[0]):
    distances[k,k] = 0.

  # max normalize
  #distances /= distances.max()
  distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

  # use tricube kernel (becaause of flatness around 0)
  probabilities = (1. - np.abs(distances) ** 3) ** 3

  # remove auto connections (which are taken anyway in LRF)
  for k in xrange(probabilities.shape[0]):
    probabilities[k,k] = 0.

  # normalize proba
  probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))
  
  # pickled for later use
  pickle.dump(probabilities, open(path,"wb"))  

In [9]:
model = Sequential()
model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=64, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(SeparableMonteCarloMaxPooling(probabilities, LRF_size=4, new_size=2500))
#model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=64, activation='relu',
#                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Conv1D(16, kernel_size=1, activation='relu',
                 padding='same',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
monte_carlo_lrf_1 (MonteCarl (None, 10000, 64)         1088      
_________________________________________________________________
separable_monte_carlo_max_po (None, 2500, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2500, 16)          1040      
_________________________________________________________________
dropout_1 (Dropout)          (None, 2500, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 40000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                800020    
Total params: 802,148
Trainable params: 802,148
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopper = EarlyStopping(min_delta=0.001, patience=2)
history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    callbacks=[early_stopper],
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10168 samples, validate on 7071 samples
Epoch 1/20
10168/10168 [==============================] - 190s - loss: 2.3104 - acc: 0.3621 - val_loss: 1.5999 - val_acc: 0.6199
Epoch 2/20
10168/10168 [==============================] - 188s - loss: 0.8276 - acc: 0.7914 - val_loss: 1.2978 - val_acc: 0.6500
Epoch 3/20
10168/10168 [==============================] - 188s - loss: 0.3765 - acc: 0.9042 - val_loss: 1.3384 - val_acc: 0.6743
Epoch 4/20
10168/10168 [==============================] - 188s - loss: 0.1908 - acc: 0.9530 - val_loss: 1.5193 - val_acc: 0.6521
Epoch 5/20
10168/10168 [==============================] - 188s - loss: 0.1076 - acc: 0.9747 - val_loss: 1.6404 - val_acc: 0.6491
Test loss: 1.6403656748
Test accuracy: 0.649130250099
